In [21]:
from numba import jit
import numpy as np

from nems.layers import DoubleExponential

# NOTE: Also tried with WC, but tensordot is not supported. After some research,
#       sounds like there would be no point to a custom implementation either
#       (b/c they end up being no better than numpy).

# NOTE: Looking like the average best-case scenario (according to others) is a
#       ~2x speedup over numpy, if there's a speedup at all, except for very
#       specific cases. So this may be useful for Layers that aren't well suited
#       for simple numpy wrappers (i.e. lots of plain python with loops), but
#       likely won't help with the main LN Layers.

pred = np.random.rand(10000, 1)
b, a, s, k = np.random.rand(4,1)
dexp = DoubleExponential(shape=(1,))
dexp.set_parameter_values(base=b, amplitude=a, shift=s, kappa=k)

class NumbaDexp:
    b = b
    a = a
    s = s
    k = k
    def evaluate(self, input):
        return _evaluate(input, self.b, self.a, self.s, self.k)

@jit(nopython=True)
def _evaluate(input, b, a, s, k):
    return b + a * np.exp(-np.exp((-np.exp(k) * (input - s))))

numba_dexp = NumbaDexp()

In [24]:
# NOTE: looks like it's even slower? may be a bad example, lots of exp.
numba_dexp.evaluate(pred)  # make sure it's compiled
%timeit dexp.evaluate(pred)
%timeit numba_dexp.evaluate(pred)

203 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
402 µs ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
